In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import List, Tuple

In [22]:
fname_ann = "/home/lorenzo/Documents/data/Test/HGD_ann.csv"
fname_data = "/home/lorenzo/Documents/data/Test/HGD.dat"
fname_weights = None

In [20]:
from aiDCA.dataset import DatasetCat, DatasetBin
import torch

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = DatasetBin(
    path_data=fname_data,
    path_labels=fname_ann,
    path_weights=fname_weights,
    device=device,
)

In [2]:
import h5py

fname_data = "/home/lorenzo/Documents/FeF_temp/example_data/MNIST_train.h5"
f = h5py.File(fname_data, "r")
print(f.keys())

<KeysViewHDF5 ['labels', 'samples']>


In [3]:
data = f["samples"][()]
labels = f["labels"][()]

In [9]:
np.savetxt("/home/lorenzo/Documents/FeF_temp/example_data/MNIST_train.dat", data, fmt="%d")

In [10]:
import pandas as pd

In [13]:
names = np.arange(len(labels)).astype(str)
df_dict = {"Name" : names, "Digit": labels}
df = pd.DataFrame(df_dict)

In [15]:
df.to_csv("/home/lorenzo/Documents/FeF_temp/example_data/MNIST_train_ann.csv", index=False)